In [2]:
import asyncio
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests

def get_chrome_options():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu") # Disable GPU acceleration
    chrome_options.add_argument("--window-size=1920x1080") # Set a default window size
    chrome_options.add_argument("--no-sandbox") # Bypass sandbox security
    chrome_options.add_argument("--user-agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'") # Set a user-agent
    return chrome_options

# # Exapmle of using the requests library to get the temperature via the OpenWeatherMap API
# def get_temperature_via_api(lat, lon, api_key):
#     url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}"
#     response = requests.get(url)
#     weather_data = response.json()
#     temperature_kelvin = weather_data['main']['temp'] # Accessing the temperature from the 'main' key
#     temperature_celsius = temperature_kelvin - 273.15 # Convert from Kelvin to Celsius
#     temperature_fahrenheit = temperature_celsius * 9/5 + 32 # Convert from Celsius to Fahrenheit
#     return temperature_fahrenheit

def get_temperature():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://weather.com/weather/tenday/l/San+Diego+CA?canonicalCityId=cb5c473781cc06501376639dce8f0823a99187dcb42c79471a4303c076d66452'
        driver.get(url)
        temperature_element = driver.find_element(By.CSS_SELECTOR, 'span.DailyContent--temp--1s3a7')
        temperature = temperature_element.text
    return temperature

def get_stock_price():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://finance.yahoo.com/quote/AAPL/'
        driver.get(url)
        stock_price_element = driver.find_element(By.CSS_SELECTOR, 'fin-streamer[data-symbol="AAPL"]')
        stock_price = stock_price_element.get_attribute('value')
    return stock_price

def get_current_time():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://www.timeanddate.com/worldclock/usa/los-angeles'
        driver.get(url)
        time_element = driver.find_element(By.ID, 'ct')
        current_time = time_element.text
    return current_time

async def main():
    loop = asyncio.get_event_loop()
    temperature = await loop.run_in_executor(None, get_temperature)
    stock_price = await loop.run_in_executor(None, get_stock_price)
    print("Temperature:", temperature)
    print("Apple Stock Price:", stock_price)

async def run_async(api_key):
    while True: # Run the loop indefinitely
        loop = asyncio.get_event_loop()
        # temperature = await loop.run_in_executor(None, get_temperature_via_api, 32.71, 117.16, api_key) #add in real api key

        temperature = await loop.run_in_executor(None, get_temperature)
        stock_price = await loop.run_in_executor(None, get_stock_price)
        current_time = await loop.run_in_executor(None, get_current_time)
        print(f"Time: {current_time} | Temperature: {round(float(temperature.split('°')[0]),2)}°F | Apple Stock Price: {stock_price}")
        
        # print(f"Time: {current_time} | Temperature: {round(float(temperature),2)}°F | Apple Stock Price: {stock_price}")

        await asyncio.sleep(60) # Wait for 60 seconds before the next iteration

In [3]:
await run_async(None)

Time: 5:34:41 pm | Temperature: 63.0°F | Apple Stock Price: 177.79
Time: 5:35:54 pm | Temperature: 63.0°F | Apple Stock Price: 177.79
Time: 5:40:17 pm | Temperature: 63.0°F | Apple Stock Price: 177.79
Time: 5:42:03 pm | Temperature: 63.0°F | Apple Stock Price: 177.79


CancelledError: 

In [3]:
import sys
from PyQt5 import QtWidgets, QtCore  # Import QtCore here
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def get_chrome_options():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--user-agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'")
    return chrome_options

def get_temperature():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://weather.com/weather/tenday/l/San+Diego+CA?canonicalCityId=cb5c473781cc06501376639dce8f0823a99187dcb42c79471a4303c076d66452'
        driver.get(url)
        temperature_element = driver.find_element(By.CSS_SELECTOR, 'span.DailyContent--temp--1s3a7')
        temperature = temperature_element.text
    return temperature

def get_stock_price():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://finance.yahoo.com/quote/AAPL/'
        driver.get(url)
        stock_price_element = driver.find_element(By.CSS_SELECTOR, 'fin-streamer[data-symbol="AAPL"]')
        stock_price = stock_price_element.get_attribute('value')
    return stock_price

def get_current_time():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://www.timeanddate.com/worldclock/usa/los-angeles'
        driver.get(url)
        time_element = driver.find_element(By.ID, 'ct')
        current_time = time_element.text
    return current_time

class App(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.timer = QtCore.QTimer()  # Use QtCore.QTimer here
        self.timer.timeout.connect(self.update_values)
        self.timer.start(60000)  # Update every 60 seconds
        self.update_values()  # Call once to initialize

    def initUI(self):
        self.label = QtWidgets.QLabel('Loading...', self)
        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.label)
        self.setLayout(layout)
        self.setWindowTitle('Weather and Stock Info')
        self.show()

    def update_values(self):
        temperature = get_temperature()
        stock_price = get_stock_price()
        current_time = get_current_time()
        text = f"Time: {current_time} | Temperature: {round(float(temperature.split('°')[0]),2)}°F | Apple Stock Price: {stock_price}"
        self.label.setText(text)

def main():
    app = QtWidgets.QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()


In [1]:
import rumps
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

def get_chrome_options():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920x1080")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--user-agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'")
    return chrome_options

def get_temperature():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://weather.com/weather/tenday/l/San+Diego+CA?canonicalCityId=cb5c473781cc06501376639dce8f0823a99187dcb42c79471a4303c076d66452'
        driver.get(url)
        temperature_element = driver.find_element(By.CSS_SELECTOR, 'span.DailyContent--temp--1s3a7')
        temperature = temperature_element.text
    return temperature

def get_stock_price():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://finance.yahoo.com/quote/AAPL/'
        driver.get(url)
        stock_price_element = driver.find_element(By.CSS_SELECTOR, 'fin-streamer[data-symbol="AAPL"]')
        stock_price = stock_price_element.get_attribute('value')
    return stock_price

def get_current_time():
    with webdriver.Chrome(options=get_chrome_options()) as driver:
        url = 'https://www.timeanddate.com/worldclock/usa/los-angeles'
        driver.get(url)
        time_element = driver.find_element(By.ID, 'ct')
        current_time = time_element.text
    return current_time

class WeatherStockApp(rumps.App):
    def __init__(self):
        super(WeatherStockApp, self).__init__("Loading...")
        self.timer = rumps.Timer(self.update_values, 60)  # Update every 60 seconds
        self.timer.start()

    @rumps.timer(60)
    def update_values(self, _):
        temperature = get_temperature()
        stock_price = get_stock_price()
        current_time = get_current_time()
        # text = f"Time: {current_time} | Temp: {round(float(temperature.split('°')[0]),2)}°F | AAPL: {stock_price}"
        text = f"Temp: {round(float(temperature.split('°')[0]),2)}°F"

        self.title = text

if __name__ == "__main__":
    app = WeatherStockApp()
    app.run()


: 

: 